In [1]:
%matplotlib inline
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
from matplotlib import pylab
from six.moves import range
from sklearn.manifold import TSNE
import MeCab
import pickle

import emoji as Emoji
from sequence import Sequence
from load_emojis import load_emojis

In [30]:
mecab = MeCab.Tagger("-Owakati")
def build_dictionary():
  dictionary = dict()
  emoji_dictionary = dict()

  dictionary['EOS'] = 0
  for sequence in Sequence.objects:
    words = mecab.parse(sequence.content).split()

    for word in words[:-1]:
      if word not in dictionary:
        dictionary[word] = len(dictionary)
    if words[-1] not in emoji_dictionary:
        emoji_dictionary[words[-1]] = len(emoji_dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
    reverse_emoji_dictionary = dict(zip(emoji_dictionary.values(), emoji_dictionary.keys())) 

  return dictionary, reverse_dictionary, emoji_dictionary, reverse_emoji_dictionary
    
dictionary, reverse_dictionary, emoji_dictionary, reverse_emoji_dictionary = build_dictionary()

print(len(dictionary))
print([word for word in dictionary][:10])
print([word_id for word_id in reverse_dictionary][:10])
print([emoji for emoji in emoji_dictionary][:10])
print([word_id for word_id in reverse_emoji_dictionary][:10])

78945
['笛', '〜〜(﹡', '忘れれ', '岩見', 'フォロバサンキュ', 'ハジカシー', '此', 'ポップングシャワー', '合', '🌉🌉🌉']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
['😂😂🍜🍥', '🍒🍊', '💖', '🏖🌻🍧🍹🍉', '︎︎🎂', '🌊👙🌞🌴', '~☕', '🙇💦💦', '🐯🐴😱', '🙂🙃']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [32]:
with open('dictionary.pickle', 'wb') as handle:
  pickle.dump(dictionary, handle)

with open('reverse_dictionary.pickle', 'wb') as handle:
  pickle.dump(reverse_dictionary, handle)

with open('emoji_dictionary.pickle', 'wb') as handle:
  pickle.dump(emoji_dictionary, handle)

with open('reverse_emoji_dictionary.pickle', 'wb') as handle:
  pickle.dump(reverse_emoji_dictionary, handle)

# with open('dictionary.pickle', 'rb') as handle:
#   dictionary = pickle.load(handle)

In [57]:
def seq2id(sequence):
  words = mecab.parse(sequence).split()
  train_ids = list(map(lambda word: dictionary[word], words[:-1]))
  label_id = emoji_dictionary[words[-1]] 
  return train_ids, label_id

sequence = Sequence.objects[100].content
train_ids, label_id = seq2id(sequence)
print(train_ids)
print(label_id)

[408, 18, 232, 240]
70


In [ ]:
lstm_size = 64

graph = tf.Graph()
with graph.as_default():
  

In [4]:
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [ skip_window ]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels

print('data:', [reverse_dictionary[di] for di in data[:8]])

for num_skips, skip_window in [(2, 1), (4, 2)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=8, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])

data: ['筋', 'UNK', 'UNK', '載せる', 'UNK', 'さき', 'な', 'おめでとう']

with num_skips = 2 and skip_window = 1:
    batch: ['UNK', 'UNK', 'UNK', 'UNK', '載せる', '載せる', 'UNK', 'UNK']
    labels: ['UNK', '筋', '載せる', 'UNK', 'UNK', 'UNK', '載せる', 'さき']

with num_skips = 4 and skip_window = 2:
    batch: ['UNK', 'UNK', 'UNK', 'UNK', '載せる', '載せる', '載せる', '載せる']
    labels: ['載せる', '筋', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'さき']


In [5]:
batch_size = 128
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 1 # How many words to consider left and right.
num_skips = 2 # How many times to reuse an input to generate a label.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default(), tf.device('/cpu:0'):

  # Input data.
  train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
  
  # Variables.
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  softmax_weights = tf.Variable(
    tf.truncated_normal([vocabulary_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
  softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Model.
  # Look up embeddings for inputs.
  embed = tf.nn.embedding_lookup(embeddings, train_dataset)
  # Compute the softmax loss, using a sample of the negative labels each time.
  loss = tf.reduce_mean(
    tf.nn.sampled_softmax_loss(softmax_weights, softmax_biases, embed,
                               train_labels, num_sampled, vocabulary_size))

  # Optimizer.
  # Note: The optimizer will optimize the softmax_weights AND the embeddings.
  # This is because the embeddings are defined as a variable quantity and the
  # optimizer's `minimize` method will by default modify all variable quantities 
  # that contribute to the tensor it is passed.
  # See docs on `tf.train.Optimizer.minimize()` for more details.
  optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
  
  # Compute the similarity between minibatch examples and all embeddings.
  # We use the cosine distance:
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
    normalized_embeddings, valid_dataset)
  similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [13]:
num_steps = 100001
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  average_loss = 0

  for step in range(num_steps):
    batch_data, batch_labels = generate_batch(
      batch_size, num_skips, skip_window)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += l
    if step % 2000 == 0:
      if step > 0:
        average_loss = average_loss / 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step %d: %f' % (step, average_loss))
      average_loss = 0
    # note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in range(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log = 'Nearest to %s:' % valid_word
        for k in range(top_k): 
          if nearest[k] < len(reverse_dictionary):
            close_word = reverse_dictionary[nearest[k]]
          else:
           close_word = nearest[k]
          log = '%s %s,' % (log, close_word)
        print(log)
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step 0: 7.659745
Nearest to 人: (❛, 転売, お好み焼き, ティッシュ, IScream, 天丼, 世, 🏊🎣,
Nearest to 1: ゲロ, 😭🙇, オフィシャル, クリフォ, 🙊🍌, 🔥🔥👧💥👩🔥, 你好, ･♡♡♡,
Nearest to う: フルート, 服部, 古野, 宿命, 拝む, (´◔‸◔`), 😦😦😦, 💓🎉🎉🎉🎉🎉💓,
Nearest to ます: 🌸💐, 変える, 悔しい, 💬🍃, 満席, 好き好き, ぽい, Second,
Nearest to 見: uUw, ง˘, 服, Fashion, 上っ面, 💃🏻💃🏻, 👗💞, 😭💛💜,
Nearest to 、: いこ, トレンディエンジェル, 面, 👯💞👯, ')╮-, 多く, 🎌💓, 了承,
Nearest to が: ManamiHPB, 莉, もうこ, 私学, 29859, 💩💩💩💩💩💘, ぉろありがとうございます, 🍶😳,
Nearest to （: 眼球, はずかし, ぃの, 😳💕💕, 直す, 🍗🍗🍗, 怖がら, 29946,
Nearest to gt: 単独, Couple, 🌸🍀, (*＾▽＾)／, 😒💛, こっ, 帰れ, 松岡,
Nearest to さん: かぶら, GENERATIONS, 29878, 熊蔵, ウリ, 発狂, 全然, 試験,
Nearest to み: 😳💔, 👧💕, フレデター, 蛸, 死ね, ぱちょんこはぎじつかいにうできないから, TouristPolice, 💟🦄,
Nearest to も: さぁ, 兼, まとまん, 涼しく, 不正, スプーン, 除目, 🌳🍃,
Nearest to いい: ChinenYuri, 🎂💖, 😎😎😎😎😎😎😎, URL, 渋い, 武道館, 再試, コイツ,
Nearest to っ: コタキ, ✌⚾, 置, ก็รู้สึกบันเทิงดี, 琴, 逮捕, Ｔｐ, 過,
Nearest to で: 白く, 遺伝子, ぴょんきちっ, 何と, リョウジ, 麻友, おしゃれ, ト,
Nearest to 💢💢💢💢: 😨😨😨, ごっちゃ, トガワ, wkwk, 落ち込ん, 憎む, (◞´•, トリプル,
Average loss 

In [1]:
import time

import numpy as np
import tensorflow as tf

import reader

In [2]:
class LSTMModel(object):
  def __init__(self, is_training):
    self.batch_size = batch_size = 20
    self.num_steps = num_steps = 20
    size = 200
    vocab_size = 20000
    
    self._input_data = tf.placeholder(tf.int32, [batch_size, num_steps])
    self._targets = tf.placeholder(tf.int32, [batch_size, num_steps])
    
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(size, forget_bias=0.0)
    cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell] * config.num_layers)
    
    self._initial_state = cell.zero_state(batch_size, tf.float32)
    
    with tf.device("/cpu:0"):
      embedding = tf.get_variable("embedding", [vocab_size, size])
    
    outputs = []
    state = self._initial_state
    with tf.variable_scope("RNN"):
      for time_step in range(num_steps):
        if time_step > 0: tf.get_variable_scope().reuse_variables()
        (cell_output, state) = cell(inputs[:, time_step, :], state)
        outputs.append(cell_output)

    output = tf.reshape(tf.concat(1, outputs), [-1, size])
    softmax_w = tf.get_variable("softmax_w", [size, vocab_size])
    softmax_b = tf.get_variable("softmax_b", [vocab_size])
    logits = tf.matmul(output, softmax_w) + softmax_b
    loss = tf.nn.seq2seq.sequence_loss_by_example(
        [logits],
        [tf.reshape(self._targets, [-1])],
        [tf.ones([batch_size * num_steps])])
    self._cost = cost = tf.reduce_sum(loss) / batch_size
    self._final_state = state
    
    if not is_training:
      return

    self._lr = tf.Variable(0.0, trainable=False)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars),
                                      config.max_grad_norm)
    optimizer = tf.train.GradientDescentOptimizer(self.lr)
    self._train_op = optimizer.apply_gradients(zip(grads, tvars))
    
  def assign_lr(self, session, lr_value):
    session.run(tf.assign(self.lr, lr_value))

  @property
  def input_data(self):
    return self._input_data

  @property
  def targets(self):
    return self._targets

  @property
  def initial_state(self):
    return self._initial_state

  @property
  def cost(self):
    return self._cost

  @property
  def final_state(self):
    return self._final_state

  @property
  def lr(self):
    return self._lr

  @property
  def train_op(self):
    return self._train_op

In [3]:
def run_epoch(session, m, data, eval_op, verbose=False):
  """Runs the model on the given data."""
  epoch_size = ((len(data) // m.batch_size) - 1) // m.num_steps
  start_time = time.time()
  costs = 0.0
  iters = 0
  state = m.initial_state.eval()
  for step, (x, y) in enumerate(reader.ptb_iterator(data, m.batch_size,
                                                    m.num_steps)):
    cost, state, _ = session.run([m.cost, m.final_state, eval_op],
                                 {m.input_data: x,
                                  m.targets: y,
                                  m.initial_state: state})
    costs += cost
    iters += m.num_steps

    if verbose and step % (epoch_size // 10) == 10:
      print("%.3f perplexity: %.3f speed: %.0f wps" %
            (step * 1.0 / epoch_size, np.exp(costs / iters),
             iters * m.batch_size / (time.time() - start_time)))

  return np.exp(costs / iters)

In [5]:
def main(_):
  raw_data = reader.ptb_raw_data()
  train_data, valid_data, test_data, _ = raw_data

  config = get_config()
  eval_config = get_config()
  eval_config.batch_size = 1
  eval_config.num_steps = 1

  with tf.Graph().as_default(), tf.Session() as session:
    initializer = tf.random_uniform_initializer(-config.init_scale,
                                                config.init_scale)
    with tf.variable_scope("model", reuse=None, initializer=initializer):
      m = PTBModel(is_training=True, config=config)
    with tf.variable_scope("model", reuse=True, initializer=initializer):
      mvalid = PTBModel(is_training=False, config=config)
      mtest = PTBModel(is_training=False, config=eval_config)

    tf.initialize_all_variables().run()

    for i in range(config.max_max_epoch):
      lr_decay = config.lr_decay ** max(i - config.max_epoch, 0.0)
      m.assign_lr(session, config.learning_rate * lr_decay)

      print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(m.lr)))
      train_perplexity = run_epoch(session, m, train_data, m.train_op,
                                   verbose=True)
      print("Epoch: %d Train Perplexity: %.3f" % (i + 1, train_perplexity))
      valid_perplexity = run_epoch(session, mvalid, valid_data, tf.no_op())
      print("Epoch: %d Valid Perplexity: %.3f" % (i + 1, valid_perplexity))

    test_perplexity = run_epoch(session, mtest, test_data, tf.no_op())
    print("Test Perplexity: %.3f" % test_perplexity)


if __name__ == "__main__":
  tf.app.run()

KeyError: '😤😤😤<eos>'